In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [2]:
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [5]:
df2 = pd.read_csv('enronSpamSubset.csv')
df2.head()

,Unnamed: 0,Unnamed: 0.1,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [7]:
df3 = pd.read_csv('lingSpam.csv')
df3.head()

,Unnamed: 0,Body,Label
0,0,Subject: great part-time or summer job !\n \n ...,1
1,1,Subject: auto insurance rates too high ?\n \n ...,1
2,2,Subject: do want the best and economical hunti...,1
3,3,Subject: email 57 million people for $ 99\n \n...,1
4,4,Subject: do n't miss these !\n \n attention ! ...,1


In [11]:
df = pd.concat([df1,df2,df3],axis=0)

In [13]:
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)

In [15]:
df['Label'].value_counts()

0    11322
1     7329
Name: Label, dtype: int64

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
def process_text(text): 
    #1 remove the punctuation 
    # 2 remove stopwords : this is useless words
    # 3 return a list of clean text words
    
    # 1 
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    # 2
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3 
    
    return clean_words

In [18]:
df['Body'].head().apply(process_text)

0    [Save, 70, Life, Insurance, Spend, ToLife, Quo...
1    [1, Fight, Risk, Cancer, httpwwwadclickwspcfmo...
2    [1, Fight, Risk, Cancer, httpwwwadclickwspcfmo...
3    [Adult, Club, Offers, FREE, Membership, INSTAN...
4    [thought, might, like, 1, Slim, Guaranteed, lo...
Name: Body, dtype: object

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['Body'])

In [27]:
(df.isna().sum() / df.shape[0]).sort_values()

Body     0.0
Label    0.0
dtype: float64

In [22]:
df.shape

(18651, 2)

In [25]:
df.dropna(axis=0,inplace=True)

In [26]:
(df.isna().sum() / df.shape[0]).sort_values()

Body     0.0
Label    0.0
dtype: float64

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow , df['Label'] , test_size=0.2)

In [63]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [72]:
rUs = RandomOverSampler()
X_under, y_under = rUs.fit_resample(message_bow , df['Label'] )

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2)

In [88]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

/Users/ahmed/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9768160741885626

In [89]:
from sklearn.model_selection import cross_val_score
cross_val_score(model,X_train,y_train,cv=3).mean()

0.9738889079947621

In [90]:
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score

In [91]:
pred = model.predict(X_test)

In [92]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2280
           1       0.96      0.99      0.98      2249

    accuracy                           0.98      4529
   macro avg       0.98      0.98      0.98      4529
weighted avg       0.98      0.98      0.98      4529



In [93]:
confusion_matrix(y_test,pred)

array([[2195,   85],
       [  20, 2229]])

1    9073
0    9042
Name: Label, dtype: int64